In [48]:
import os
import json
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [49]:
count_folder = '../../results-demo/public/data/count-only/'
nn_as_metrics_folder = '../../results-demo/public/data/param-nn-ne-0.5'
ee_as_metrics_folder = '../../results-demo/public/data/param-ee-quadratic-0.20/'

In [50]:
graphs_file_path = os.path.join(nn_as_metrics_folder, 'graphs.txt')
graph_names = [l.strip() for l in open(graphs_file_path)]

In [51]:
df_arr = []

# Extract the computational times
for g in graph_names:
    res_path = os.path.join(count_folder, g + '_result.json')
    res_data_count = json.load(open(res_path))
    
    res_path = os.path.join(nn_as_metrics_folder, g + '_result.json')
    res_data_nn = json.load(open(res_path))

    res_path = os.path.join(ee_as_metrics_folder, g + '_result.json')
    res_data_ee = json.load(open(res_path))

    df_arr.append({
        'layout': g,
        'count-nn': res_data_count['metrics']['nn']['execution_time'],
        'count-ne': res_data_count['metrics']['ne']['execution_time'],
        'count-ee': res_data_count['metrics']['ee']['execution_time'],
        'as-nn': res_data_nn['metrics']['nn']['execution_time'],
        'as-ne': res_data_nn['metrics']['ne']['execution_time'],
        'as-ee': res_data_ee['metrics']['ee']['execution_time']
    })

In [52]:
d = pd.DataFrame(df_arr, columns=['layout', 'as-nn', 'count-nn', 'as-ne', 'count-ne', 'as-ee', 'count-ee'])

In [53]:
d['ratio-nn'] = d['as-nn'] / d['count-nn']
d['ratio-ne'] = d['as-ne'] / d['count-ne']
d['ratio-ee'] = d['as-ee'] / d['count-ee']

In [54]:
d

,layout,as-nn,count-nn,as-ne,count-ne,as-ee,count-ee,ratio-nn,ratio-ne,ratio-ee
0,four-clusters-original,0.011802,0.006418,0.078348,0.037459,0.120321,0.058008,1.838891,2.091567,2.074214
1,four-clusters-ne0,0.012194,0.005628,0.083977,0.042639,0.140157,0.057877,2.166667,1.969488,2.421636
2,four-clusters-ne1,0.012123,0.005778,0.082959,0.046403,0.172271,0.071535,2.098131,1.787794,2.408206
3,four-clusters-nn0,0.017778,0.007352,0.087637,0.062523,0.129094,0.093032,2.418118,1.401676,1.387630
4,four-clusters-nn1,0.011881,0.007919,0.086108,0.053095,0.142471,0.088915,1.500316,1.621772,1.602328
5,four-clusters-nn2,0.012599,0.006347,0.085074,0.052182,0.165360,0.071242,1.985032,1.630332,2.321103
6,four-clusters-ee0,0.012379,0.008601,0.094860,0.049304,0.208847,0.077762,1.439251,1.923982,2.685721
7,special-cases-1,0.002847,0.001717,0.012307,0.007330,0.013876,0.005609,1.658125,1.678990,2.473881
8,special-cases-2,0.002723,0.001371,0.011852,0.006299,0.014793,0.006229,1.986142,1.881569,2.374860
9,special-cases-3,0.008903,0.008032,0.085639,0.052077,0.122468,0.071306,1.108441,1.644469,1.717499


In [55]:
d[d['count-nn'] > 1.0]['ratio-nn'].describe()

count    6.000000
mean     1.344897
std      0.084188
min      1.211784
25%      1.304032
50%      1.360034
75%      1.393740
max      1.447190
Name: ratio-nn, dtype: float64

In [62]:
d[d['count-ne'] > 1]['ratio-ne'].describe()

count    10.000000
mean      1.485260
std       0.418273
min       1.010149
25%       1.269163
50%       1.360447
75%       1.561708
max       2.483624
Name: ratio-ne, dtype: float64

In [57]:
d[d['count-ee'] > 1.0]['ratio-ee'].describe()

count    14.000000
mean      2.599039
std       0.481791
min       1.972894
25%       2.135097
50%       2.543231
75%       3.042311
max       3.367639
Name: ratio-ee, dtype: float64